In [8]:
import pandas as pd 
import numpy as np 

X = pd.read_csv('../Boosts/Datasets/X_train.csv')
y = pd.read_csv('../Boosts/Datasets/y_train.csv')
X_valid = pd.read_csv('../Boosts/Datasets/X_test.csv')
valid_user_id = pd.read_csv('../Boosts/Datasets/test_user_id.csv')

In [9]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.75, random_state=123)

In [10]:
import joblib 

rf = joblib.load('../Boosts/BoostedModels/boosted_random_forest.joblib')
rf

RandomForestClassifier(bootstrap=False, class_weight={0: 2, 1: 3},
                       max_features=10, min_samples_leaf=6,
                       min_samples_split=6)

In [11]:
rf_pred = rf.predict(X_test)

In [12]:
pd.DataFrame(classification_report(y_test, rf_pred, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.888645,0.774199,0.841192,0.831422,0.844447
recall,0.847468,0.831217,0.841192,0.839343,0.841192
f1-score,0.867568,0.801696,0.841192,0.834632,0.842129
support,8136.000000,5119.000000,0.841192,13255.000000,13255.000000


In [13]:
from sklearn.neural_network import MLPClassifier
dnn = MLPClassifier(hidden_layer_sizes=(32,16), max_iter=500, random_state=42)
dnn.fit(X_train, np.ravel(y_train))

MLPClassifier(hidden_layer_sizes=(32, 16), max_iter=500, random_state=42)

In [14]:
dnn_preds = dnn.predict(X_test)
pd.DataFrame(classification_report(y_test, dnn_preds, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.833847,0.771642,0.811241,0.802744,0.809824
recall,0.864798,0.726118,0.811241,0.795458,0.811241
f1-score,0.849041,0.748188,0.811241,0.798615,0.810092
support,8136.000000,5119.000000,0.811241,13255.000000,13255.000000


In [15]:
from sklearn.ensemble import VotingClassifier
ensembled_dnn = VotingClassifier(estimators=[('rf', rf), ('dnn', dnn)], voting='soft')
ensembled_dnn.fit(X_train, np.ravel(y_train))

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=False,
                                                     class_weight={0: 2, 1: 3},
                                                     max_features=10,
                                                     min_samples_leaf=6,
                                                     min_samples_split=6)),
                             ('dnn',
                              MLPClassifier(hidden_layer_sizes=(32, 16),
                                            max_iter=500, random_state=42))],
                 voting='soft')

In [16]:
ensembled_dnn_preds = ensembled_dnn.predict(X_test)
pd.DataFrame(classification_report(y_test, ensembled_dnn_preds, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.851959,0.787959,0.82814,0.819959,0.827242
recall,0.871436,0.759328,0.82814,0.815382,0.828140
f1-score,0.861587,0.773378,0.82814,0.817483,0.827521
support,8136.000000,5119.000000,0.82814,13255.000000,13255.000000


In [17]:
joblib.dump(ensembled_dnn, '../Boosts/BoostedModels/MLPC_AdaBoost.joblib', compress=('gzip',9))

['../Boosts/BoostedModels/MLPC_AdaBoost.joblib']

<hr>

### Submission

In [18]:
ensembled_dnn.fit(X, np.ravel(y))

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=False,
                                                     class_weight={0: 2, 1: 3},
                                                     max_features=10,
                                                     min_samples_leaf=6,
                                                     min_samples_split=6)),
                             ('dnn',
                              MLPClassifier(hidden_layer_sizes=(32, 16),
                                            max_iter=500, random_state=42))],
                 voting='soft')

In [20]:
rf_mlpc_submission = ensembled_dnn.predict(X_valid)

In [26]:
rf.fit(X, np.ravel(y))
rf_submission = rf.predict(X_valid)

In [23]:
valid_user_id['moved_after_2019'] = rf_mlpc_submission; valid_user_id.head(3)

,user_id,moved_after_2019
0,17449,0
1,33967,0
2,2110,0


In [24]:
valid_user_id.to_csv('../Boosts/Submissions/rf_mlpc_submission.csv', index=False)

In [27]:
rf_submission_df = valid_user_id
rf_submission_df['moved_after_2019'] = rf_submission; rf_submission_df.head()

,user_id,moved_after_2019
0,17449,0
1,33967,0
2,2110,0
3,55082,0
4,37165,0


In [29]:
rf_submission_df.to_csv('../Boosts/Submissions/rf_submission.csv', index=False)